In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emnist/emnist-letters-mapping.txt
/kaggle/input/emnist/emnist-letters-test.csv
/kaggle/input/emnist/emnist-mnist-mapping.txt
/kaggle/input/emnist/emnist-digits-train.csv
/kaggle/input/emnist/emnist-bymerge-mapping.txt
/kaggle/input/emnist/emnist-balanced-train.csv
/kaggle/input/emnist/emnist-digits-test.csv
/kaggle/input/emnist/emnist-balanced-test.csv
/kaggle/input/emnist/emnist-mnist-test.csv
/kaggle/input/emnist/emnist-letters-train.csv
/kaggle/input/emnist/emnist-byclass-train.csv
/kaggle/input/emnist/emnist-bymerge-test.csv
/kaggle/input/emnist/emnist-balanced-mapping.txt
/kaggle/input/emnist/emnist-mnist-train.csv
/kaggle/input/emnist/emnist-digits-mapping.txt
/kaggle/input/emnist/emnist-bymerge-train.csv
/kaggle/input/emnist/emnist-byclass-test.csv
/kaggle/input/emnist/emnist-byclass-mapping.txt
/kaggle/input/emnist/emnist_source_files/emnist-digits-test-labels-idx1-ubyte
/kaggle/input/emnist/emnist_source_files/emnist-bymerge-train-labels-idx1-ubyte
/kaggle/input/

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("crawford/emnist")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/emnist


In [4]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from scipy.stats import pearsonr
import warnings
warnings.filterwarnings('ignore')

# Check TensorFlow version and GPU availability
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"Keras version: {keras.__version__}")

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

TensorFlow version: 2.18.0
GPU Available: []
Keras version: 3.8.0


2025-09-29 20:13:34.520331: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
# Load EMNIST dataset from local path
import os
import pandas as pd
import numpy as np

print("Loading EMNIST dataset from local path...")

# Path to the EMNIST dataset
dataset_path = path

try:
    # List files in the dataset directory
    files = os.listdir(dataset_path)
    print(f"Files in dataset directory: {files}")
    
    # Look for EMNIST ByClass files (62 classes: 0-9, A-Z, a-z)
    train_file = None
    test_file = None
    
    for file in files:
        if 'byclass' in file.lower() and 'train' in file.lower():
            train_file = file
        elif 'byclass' in file.lower() and 'test' in file.lower():
            test_file = file
    
    if train_file and test_file:
        print(f"Found training file: {train_file}")
        print(f"Found test file: {test_file}")
        
        # Load the CSV files
        train_data = pd.read_csv(os.path.join(dataset_path, train_file))
        # Filter to include only classes 0-35 (digits 0-9 and uppercase A-Z)
        train_data = train_data[train_data.iloc[:, 0].isin(np.arange(0, 36))]
        test_data = pd.read_csv(os.path.join(dataset_path, test_file))
        test_data = test_data[test_data.iloc[:, 0].isin(np.arange(0, 36))]
        
        print(f"Training data shape: {train_data.shape}")
        print(f"Test data shape: {test_data.shape}")
        
        # Extract features and labels
        # First column is label, rest are pixel values
        x_train = train_data.iloc[:, 1:].values
        y_train = train_data.iloc[:, 0].values
        x_test = test_data.iloc[:, 1:].values
        y_test = test_data.iloc[:, 0].values
        
        # Reshape from flat arrays to 28x28 images
        x_train = x_train.reshape(-1, 28, 28)
        x_test = x_test.reshape(-1, 28, 28)
        
        # EMNIST images need to be rotated and flipped to match standard orientation
        x_train = np.rot90(x_train, k=3, axes=(1, 2))  # Rotate 270 degrees
        x_test = np.rot90(x_test, k=3, axes=(1, 2))    # Rotate 270 degrees
        x_train = np.flip(x_train, axis=2)  # Flip horizontally
        x_test = np.flip(x_test, axis=2)    # Flip horizontally
        
        num_classes = len(np.unique(y_train))
        
        print(f"\nEMNIST ByClass Dataset loaded successfully!")
        print(f"Training data shape: {x_train.shape}")
        print(f"Training labels shape: {y_train.shape}")
        print(f"Test data shape: {x_test.shape}")
        print(f"Test labels shape: {y_test.shape}")
        print(f"Number of classes: {num_classes}")
        print(f"Label range: {np.min(y_train)} to {np.max(y_train)}")
        
        # Create class mapping
        class_names = []
        # Classes 0-9: Digits
        class_names.extend([str(i) for i in range(10)])
        # Classes 10-35: Uppercase letters A-Z
        class_names.extend([chr(ord('A') + i) for i in range(26)])
        
        print(f"Class mapping: 0-9 (digits), 10-35 (A-Z)")
        print(f"Sample classes: {class_names[:10]}...{class_names[-10:]}")
        
    else:
        raise FileNotFoundError("Could not find ByClass train/test files")

except Exception as e:
    print(f"Error loading dataset: {e}")
    raise e

Loading EMNIST dataset from local path...
Files in dataset directory: ['emnist-letters-mapping.txt', 'emnist-letters-test.csv', 'emnist-mnist-mapping.txt', 'emnist-digits-train.csv', 'emnist-bymerge-mapping.txt', 'emnist-balanced-train.csv', 'emnist-digits-test.csv', 'emnist-balanced-test.csv', 'emnist-mnist-test.csv', 'emnist-letters-train.csv', 'emnist-byclass-train.csv', 'emnist-bymerge-test.csv', 'emnist-balanced-mapping.txt', 'emnist-mnist-train.csv', 'emnist-digits-mapping.txt', 'emnist-bymerge-train.csv', 'emnist-byclass-test.csv', 'emnist_source_files', 'emnist-byclass-mapping.txt']
Found training file: emnist-byclass-train.csv
Found test file: emnist-byclass-test.csv
Training data shape: (533992, 785)
Test data shape: (89263, 785)

EMNIST ByClass Dataset loaded successfully!
Training data shape: (533992, 28, 28)
Training labels shape: (533992,)
Test data shape: (89263, 28, 28)
Test labels shape: (89263,)
Number of classes: 36
Label range: 0 to 35
Class mapping: 0-9 (digits), 1

In [ ]:
# Image Preprocessing
print("Original data shapes:")
print(f"x_train: {x_train.shape}, x_test: {x_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")
print(f"Pixel value range: {x_train.min()} to {x_train.max()}")

# 1. Normalization: Scale pixel values to [0, 1] range
# Formula: I_norm = I/255
x_train_norm = x_train.astype('float32') / 255.0
x_test_norm = x_test.astype('float32') / 255.0

print(f"\nAfter normalization:")
print(f"Pixel value range: {x_train_norm.min()} to {x_train_norm.max()}")

# 2. Reshaping for CNN input: Add channel dimension
# From (28, 28) to (28, 28, 1)
x_train_reshaped = x_train_norm.reshape(x_train_norm.shape[0], 28, 28, 1)
x_test_reshaped = x_test_norm.reshape(x_test_norm.shape[0], 28, 28, 1)

print(f"\nAfter reshaping:")
print(f"x_train: {x_train_reshaped.shape}, x_test: {x_test_reshaped.shape}")


In [ ]:
# 3. Label encoding: Convert to one-hot encoded vectors
y_train_onehot = keras.utils.to_categorical(y_train, num_classes)
y_test_onehot = keras.utils.to_categorical(y_test, num_classes)

print(f"\nAfter one-hot encoding:")
print(f"y_train: {y_train_onehot.shape}, y_test: {y_test_onehot.shape}")
print(f"Example label before: {y_train[0]}")
print(f"Example label after: {y_train_onehot[0]}")

# Store preprocessed data
x_train_processed = x_train_reshaped
x_test_processed = x_test_reshaped
y_train_processed = y_train_onehot
y_test_processed = y_test_onehot

In [ ]:
# Visualize sample images from the dataset
plt.figure(figsize=(12, 8))
for i in range(20):
    plt.subplot(4, 5, i + 1)
    plt.imshow(x_train[i], cmap='gray')
    plt.title(f'Label: {y_train[i]}')
    plt.axis('off')

plt.suptitle('Sample Images from MNIST Dataset', fontsize=16)
plt.tight_layout()
plt.show()

# Show distribution of classes
plt.figure(figsize=(10, 6))
unique, counts = np.unique(y_train, return_counts=True)
plt.bar(unique, counts)
plt.title('Distribution of Classes in Training Set')
plt.xlabel('Digit Class')
plt.ylabel('Number of Samples')
plt.xticks(unique)
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Define CNN Architecture for Extended Alphanumeric Dataset
def create_cnn_model(input_shape=(28, 28, 1), num_classes=62):
    """
    Create CNN model according to requirements specifications
    Updated for 62 classes (0-9, A-Z, a-z)
    """
    model = keras.Sequential([
        # Input Layer
        keras.Input(shape=input_shape),
        
        # Layer 1: Conv2D with 32 filters, 3x3 kernel, ReLU activation
        layers.Conv2D(32, (3, 3), activation='relu', padding='valid', name='conv2d_1'),
        
        # Layer 2: MaxPooling2D with 2x2 pool size
        layers.MaxPooling2D((2, 2), name='maxpool2d_1'),
        
        # Layer 3: Conv2D with 64 filters, 3x3 kernel, ReLU activation
        layers.Conv2D(64, (3, 3), activation='relu', padding='valid', name='conv2d_2'),
        
        # Layer 4: MaxPooling2D with 2x2 pool size
        layers.MaxPooling2D((2, 2), name='maxpool2d_2'),
        
        # Layer 5: Flatten layer (feature extractor)
        layers.Flatten(name='flatten'),
        
        # Layer 6: Dense layer with 128 units, ReLU activation
        layers.Dense(128, activation='relu', name='dense_1'),
        
        # Layer 7: Output Dense layer with 62 classes for alphanumeric characters
        layers.Dense(num_classes, activation='softmax', name='output')
    ])
    
    return model

# Create the model with 62 classes
cnn_model = create_cnn_model(input_shape=(28, 28, 1), num_classes=num_classes)

# Display model architecture
print("CNN Model Architecture for Extended Alphanumeric Dataset:")
print(f"Number of classes: {num_classes}")
cnn_model.summary()


In [ ]:
# Show the class mapping
print(f"\nClass Mapping:")
print(f"0-9: Digits")
print(f"10-35: Uppercase letters (A-Z)")
print(f"Total classes: {num_classes}")

In [ ]:
# Compile the CNN model
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully!")
print("Optimizer: Adam")
print("Loss Function: Categorical Cross-Entropy")
print("Metrics: Accuracy")


# Train the model
print("\nStarting CNN training...")
history = cnn_model.fit(
    x_train_processed, y_train_processed,
    batch_size=16,
    epochs=20,
    validation_split=0.2,
    verbose=1
)

print("\nCNN training completed!")

In [ ]:
# Plot training history
plt.figure(figsize=(15, 5))

# Plot training & validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final training results
print("Final Training Results:")
print(f"Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Validation Loss: {history.history['val_loss'][-1]:.4f}")

In [ ]:
# Evaluate Standalone CNN Performance
print("Evaluating CNN on Test Set...")
print("=" * 50)

# Get CNN predictions on test set
cnn_predictions = cnn_model.predict(x_test_processed, batch_size=32, verbose=1)
cnn_predicted_classes = np.argmax(cnn_predictions, axis=1)

# Calculate CNN metrics
cnn_correct_predictions = np.sum(cnn_predicted_classes == y_test)
cnn_accuracy = cnn_correct_predictions / len(y_test)
cnn_error_rate = 1 - cnn_accuracy

print(f"\nStandalone CNN Results:")
print(f"Correct predictions: {cnn_correct_predictions}/{len(y_test)}")
print(f"Accuracy: {cnn_accuracy:.4f} ({cnn_accuracy*100:.2f}%)")
print(f"Error Rate: {cnn_error_rate:.4f} ({cnn_error_rate*100:.2f}%)")